In [ ]:
! pip install ../input/convmodel/Keras_Applications-1.0.8-py3-none-any.whl
! pip install ../input/convmodel/efficientnet-1.1.1-py3-none-any.whl

In [ ]:
import os
import cv2
import pydicom
from pydicom import dcmread
import matplotlib.pyplot as plt
import numpy as np
import random
from numpy import dstack
from scipy.ndimage import zoom
import skimage
import tensorflow as tf
from tensorflow_addons.losses import pinball_loss
from tensorflow_addons.optimizers import RectifiedAdam
from tensorflow.keras.optimizers import Nadam
from PIL import Image
from tensorflow.keras.backend import greater, zeros_like, variable, constant, int_shape, mean, shape, ones_like
from tensorflow import boolean_mask, where
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical, Sequence
from pickle import load
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,LSTM,TimeDistributed, Input,BatchNormalization, Activation, Conv3D,MaxPooling3D, Dropout, concatenate,GlobalAveragePooling2D,Conv2D,AveragePooling2D,LeakyReLU,Concatenate
from keras.models import Sequential, Model
from tensorflow.math import abs, sqrt, minimum, maximum, log, multiply, divide_no_nan, add
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import seaborn as sns
from tqdm.notebook import tqdm 
from sklearn.model_selection import train_test_split

from shutil import copyfile,copy
copyfile(src = "../input/convmodel/applinearmodel/src/linear_model.py", dst = "../working/linear_model.py")
copyfile(src = "../input/convmodel/convmdlnosave.py", dst = "../working/convmdlnosave.py")
copyfile(src = "../input/convmodel/EfficientNet_Pred_v1.1.py", dst = "../working/EfficientNet_Pred.py")
copyfile(src = "../input/convmodel/fold-2_best.h5", dst = "../working/fold-2_best.h5")

from linear_model import NoTransformer, load_huber_models, huber_predict
from convmdlnosave import conv3dModel
from EfficientNet_Pred import start_predict
from tensorflow.keras.layers import (
    Dense, Dropout, Activation, Flatten, Input, BatchNormalization, GlobalAveragePooling2D, Add, Conv2D, AveragePooling2D, 
    LeakyReLU, Concatenate 
)
import efficientnet.tfkeras as efn

# Defining Functions

In [ ]:
def pinball2(y_true, y_pred):
  zeros = zeros_like(y_true)
  bool_mask = greater(y_true, [0])
  y_pred = where(bool_mask, y_pred, y_true)
  #tf.print(y_pred, summarize = 20)

  return pinball_loss(y_true, y_pred)

def LLL_metric2(y_true, y_pred):
  zeros = zeros_like(y_true)
  bool_mask = greater(y_true, [0])
  y_pred = where(bool_mask, y_pred, y_true)

  diff = abs(y_pred - y_true)
  sigma = constant(value = 200, shape = [146])

  delta = minimum(diff, constant(value = 1000, shape = [146]))
  delta = diff
  sqrt2 = constant(value = 1.414, shape = [146])
  
  loss = - divide_no_nan(sqrt2 * delta, sigma) - log(where(bool_mask, sqrt2 * sigma, ones_like(y_true)))
  avg_loss = mean(boolean_mask(loss, bool_mask), axis = 0)
  
  return avg_loss

# Chun How Model

# Data Prep

In [ ]:
import numpy as np
a=np.array([6,6,6])
b=np.array([4,3,2])
a-b

# Ensemble

In [ ]:
# create stacked model input dataset as outputs from the ensemble
# Returns the output of all the different models to be used as the inputs for the ensemble
def stacked_dataset(inputX):
    stacked = None
    stackX = []
    # make prediction
    failed=[]
    stackConf=[]
    for i in range(len(inputX)):
        print(i)
        age=inputX[i][0]
        sex=inputX[i][1]
        week=inputX[i][2]
        fvc=inputX[i][3]
        percent=inputX[i][4]
        smokingstatus=inputX[i][5]
        imgpath=inputX[i][6]+'/'
        modelpath=inputX[i][7]
        scalerpath=inputX[i][8]
        patient=inputX[i][9]
        try:
            convMdl=conv3dModel(age,sex,week,fvc,percent,smokingstatus,imgpath,modelpath,scalerpath)
            effNetMdlOut=start_predict(fvc,week,percent,age,sex,smokingstatus,imgpath)
            # initialize models
            effNetMdl=[i[0] for i in effNetMdlOut[1::3]]
            effNetConf=[i[0] for i in effNetMdlOut[2::3]]
            lower_huber, mid_huber, upper_huber, datawrangler = load_huber_models('../input/convmodel/applinearmodel/model_weights/linear_model/lower_huber.pkl',
                                                                                '../input/convmodel/applinearmodel/model_weights/linear_model/mid_huber.pkl',
                                                                                '../input/convmodel/applinearmodel/model_weights/linear_model/upper_huber.pkl',                                                                                '../input/convmodel/applinearmodel/model_weights/linear_model/datawrangler.pkl')

            # prediction for linear model
            df = huber_predict(lower_huber, mid_huber, upper_huber, datawrangler, patient, week, fvc, percent, age, sex, smokingstatus)
            df = df[['FVC','Lower','Upper']]

            huberMdl=list(df.FVC)
            huberConf=np.abs(df.Lower-df.Upper)
            confTotal=(np.array(effNetConf)+huberConf)/2
            stacked = dstack((convMdl,huberMdl,effNetMdl))
            stackX.append(stacked)
            stackConf.append(confTotal)
        except:
            failed.append(i)
            continue
    stackX=np.array([i[0] for i in stackX])
    return stackX,stackConf,failed

 
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(inputX, inputy,testX,testy):
    model=create_stacked_model()
    # create dataset using ensemble
    stackedX,failed = stacked_dataset(inputX)
    stackedXtest,failedtest = stacked_dataset(inputX)
    inputy=[inputy[i] for i in range(len(inputy)) if i not in failed]
    testy=[texty[i] for i in range(len(texty)) if i not in failedtest]
    # fit standalone model
    model.fit(stackedX, inputy,validation_data=(stackedXtest,testy), epochs=10)
    return model
 
# make a prediction with the stacked model
def stacked_prediction(inputX):
    model=tf.keras.models.load_model('../input/convmodel/mkEnsemble.hdf5',compile=False)
    # create dataset using ensemble
    stackedX,conf,failed = stacked_dataset(inputX)
    # make a prediction
    yhat = model.predict(np.array(stackedX))
    return yhat,conf

def create_stacked_model():
    inputs=Input(shape=(146,3),name="ensemble_input")
    flatten=Flatten()(inputs)
    dense1=Dense(256, activation='relu')(flatten)
    dense2=Dense(256, activation='relu')(dense1)
    outputs=Dense(146, activation='linear',name="FVC")(dense2)
    model=Model(inputs=inputs,outputs=outputs,name="stacked_model")
    model.compile(loss=pinball2, optimizer='adam', metrics=LLL_metric2)
    return model

def createLine(outPred):
    click=0
    first=0
    last=0
    for i in range(len(outPred)):
        if outPred[i]<=1001 and click==0:
            first=i+1
        elif outPred[i]<=1001 and click==1 and i>20:
            last=i+1
            break
        if not (outPred[i]<=1001):
            click=1

    y=np.array(outPred[first:last])
    x=np.array(list(range(first-12,last-12)))
    xmean=np.mean(x)
    ymean=np.mean(y)

    xycov = (x - xmean) * (y - ymean)
    xvar = (x - xmean)**2
    beta = xycov.sum() / xvar.sum()
    alpha = ymean - (beta * xmean)
    line=[i*beta+alpha for i in np.array(list(range(-12,134)))]
    return line

# Data Prep to test

In [ ]:
import pandas as pd
loc="../input/osic-pulmonary-fibrosis-progression/"
my_data = pd.read_csv(loc+'test.csv', delimiter=',')
my_data['imgloc']="/kaggle/input/osic-pulmonary-fibrosis-progression/test/"+my_data.Patient
age=list(my_data.Age)
sex=list(my_data.Sex)
week=list(my_data.Weeks)

cat_list = []
merge =my_data
final_df = pd.merge(merge.groupby('Patient')['Weeks'].apply(list).to_frame(),
                    merge.groupby('Patient')['FVC'].apply(list).to_frame(),
                    on = 'Patient')
final_df = pd.merge(final_df, merge.groupby('Patient')['Percent'].apply(list).to_frame(), on = 'Patient')
final_df = pd.merge(final_df, merge.groupby('Patient')['Age'].first().to_frame(), on = 'Patient')
final_df = pd.merge(final_df, merge.groupby('Patient')['Sex'].first().to_frame(), on = 'Patient')
final_df = pd.merge(final_df, merge.groupby('Patient')['SmokingStatus'].first().to_frame(), on = 'Patient')
final_df = pd.merge(final_df, merge.groupby('Patient')['imgloc'].first().to_frame(), on = 'Patient')
final_df['modelpath']="../input/convmodel/"
final_df['scalerpath']="../input/convmodel/"
final_df = final_df.reset_index()

y = np.zeros((final_df.shape[0], 146))

for index, row in final_df.iterrows():
    y[index][np.array(row['Weeks']) + 12] = row['FVC']
    
age=list(final_df.Age)
sex=list(final_df.Sex)
week=list(final_df.Weeks[0])
fvc=list(final_df.FVC[0])
percent=list(final_df.Percent)
smokingstatus=list(final_df.SmokingStatus)
imgpath=list(final_df.imgloc)
modelpath=list(final_df.modelpath)
scalerpath=list(final_df.scalerpath)
patient=list(final_df.Patient)


inputs = []
for index, row in final_df.iterrows():
    x1=[]
    x1.append(row['Age'])
    x1.append(row['Sex'])
    x1.append(row['Weeks'][0] + 12)
    x1.append(row['FVC'][0])
    x1.append(row['Percent'][0])
    x1.append(row['SmokingStatus'])
    x1.append(row['imgloc'])
    x1.append("../input/convmodel/")
    x1.append("../input/convmodel/")
    x1.append(row['Patient'])
    inputs.append(x1)

In [ ]:
#Model Callback  # Step 1
modelname='mkEnsemble'
folderpath      = '../working/'
filepath        = folderpath + "mkEnsemble.hdf5"
checkpoint      = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                  monitor='val_LLL_metric2', 
                                  verbose=0, 
                                  save_best_only=True, 
                                  mode='max')

csv_logger      = tf.keras.callbacks.CSVLogger(folderpath+modelname +'.csv')                       # Step 2
callbacks_list  = [checkpoint,csv_logger]                                       # Step 3

print("Callbacks created:")
print(callbacks_list[0])
print(callbacks_list[1])
print('')
print("Path to model:", filepath)
print("Path to log:  ", folderpath+modelname+'.csv')
#keras.utils.plot_model(model3, "multi_input_and_output_model.png", show_shapes=True)

In [ ]:
X=inputs
y=y
#out=stacked_prediction([X[1]])


In [ ]:
submission=pd.DataFrame()
for i in range(len(y)): #len(y)
    outPred,confidenceList=stacked_prediction([X[i]])
    outPred=np.array(outPred[0])
    patientList=[]
    line=createLine(outPred)
    outPred2=[outPred[i] if np.abs(outPred[i]-line[i])<300 else line[i] for i in range(len(outPred))]
    for j in range(len(outPred2)):
        patientList.append(final_df.Patient[i]+'_'+str(j-12))
    data=[]
    for j in range(len(outPred2)):
        data.append([patientList[j],outPred2[j],np.round(confidenceList[0][j])])
        
    newDF=pd.DataFrame(data,columns=['Patient_Week','FVC','Confidence'])
    if i==0:
        submission=newDF
        print(0)
    else:
        submission=submission.append(newDF)
        print(i)

In [ ]:
submission.to_csv('submission.csv',index=False)